In [116]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score

from multi_imbalance.ensemble.ovo import OVO
from multi_imbalance.datasets import load_datasets

In [117]:
minority = dict()
minority['1czysty-cut'] = [1, 2]
minority['2delikatne-cut'] = [1, 2]
minority['3mocniej-cut'] = [1, 2]
minority['4delikatne-bezover-cut'] = [1, 2]
minority['balance-scale'] = [0]
minority['cleveland'] = [4, 3, 2]
minority['cleveland_v2'] = [3, 2, 1]
minority['cmc'] = [1]
minority['dermatology'] = [5, 3, 4, 1]
minority['glass'] = [4, 2, 5, 3]
minority['hayes-roth'] = [0]
minority['new_ecoli'] = [3, 2, 4]
minority['new_led7digit'] = [1, 4]
minority['new_vehicle'] = [2, 0]
minority['new_winequality-red'] = [3, 2]
minority['new_yeast'] = [2, 3, 4, 5, 6]
minority['thyroid-newthyroid'] = [2, 1]

In [118]:
datasets = load_datasets()
datasets_names = [dsn for dsn, _ in datasets.items()]

binary_classifiers = ['CART', 'KNN', 'NB']
binary_oversamplings = [None, 'globalCS', 'SMOTE']
oversample_between_strategies = ['all', 'maj-min']

results_g_mean = dict()
results_acc = dict()
results_avg_tpr = dict()

methods=[(bc,bo,obs) for bc in binary_classifiers for bo in binary_oversamplings for obs in oversample_between_strategies]

for res in (results_g_mean, results_acc, results_avg_tpr):
    res['dataset'] = datasets_names
    for method in methods:
        res[method] = list()

In [119]:
for dataset_name, dataset_values in datasets.items():
    X, y = dataset_values.data, dataset_values.target

    for binary_classifier in binary_classifiers:
        for oversample_binary in binary_oversamplings:
            for oversample_between_strategy in oversample_between_strategies:

                print(dataset_name, oversample_binary, binary_classifier, oversample_between_strategy)
                acc, g_mean, avg_tpr = list(), list(), list()

                for i in range(10):
                    fold_no = 0
                    skf = StratifiedKFold(n_splits=5, random_state=i)

                    for train_index, test_index in skf.split(X, y):
                        X_train, X_test = X[train_index], X[test_index]
                        y_train, y_test = y[train_index], y[test_index]

                        nn = min(np.unique(y_train, return_counts=True)[1]) - 1
                        nn = min(nn, 3)
                        if nn == 2: nn = 1

                        ovo = OVO(binary_classifier=binary_classifier, oversample_binary=oversample_binary, n_neighbors=nn,
                                  oversample_between=oversample_between_strategy)
                        ovo.fit(X_train, y_train, minority_classes=minority[dataset_name])
                        y_pred = ovo.predict(X_test)

                        g_mean.append(geometric_mean_score(y_test, y_pred, correction=0.001))
                        acc.append(accuracy_score(y_test, y_pred))
                        avg_tpr.append(recall_score(y_test, y_pred, average='macro'))

                        fold_no += 1


                results_g_mean[(binary_classifier, oversample_binary, oversample_between_strategy)].append(round(np.mean(g_mean), 3))
                results_acc[(binary_classifier, oversample_binary, oversample_between_strategy)].append(round(np.mean(acc), 3))
                results_avg_tpr[(binary_classifier, oversample_binary, oversample_between_strategy)].append(round(np.mean(avg_tpr), 3))


1czysty-cut None CART all
1czysty-cut None CART maj-min
1czysty-cut globalCS CART all
1czysty-cut globalCS CART maj-min
1czysty-cut SMOTE CART all
1czysty-cut SMOTE CART maj-min
1czysty-cut None KNN all
1czysty-cut None KNN maj-min
1czysty-cut globalCS KNN all
1czysty-cut globalCS KNN maj-min
1czysty-cut SMOTE KNN all
1czysty-cut SMOTE KNN maj-min
1czysty-cut None NB all
1czysty-cut None NB maj-min
1czysty-cut globalCS NB all
1czysty-cut globalCS NB maj-min
1czysty-cut SMOTE NB all
1czysty-cut SMOTE NB maj-min
2delikatne-cut None CART all
2delikatne-cut None CART maj-min
2delikatne-cut globalCS CART all
2delikatne-cut globalCS CART maj-min
2delikatne-cut SMOTE CART all
2delikatne-cut SMOTE CART maj-min
2delikatne-cut None KNN all
2delikatne-cut None KNN maj-min
2delikatne-cut globalCS KNN all
2delikatne-cut globalCS KNN maj-min
2delikatne-cut SMOTE KNN all
2delikatne-cut SMOTE KNN maj-min
2delikatne-cut None NB all
2delikatne-cut None NB maj-min
2delikatne-cut globalCS NB all
2delikatn

new_yeast SMOTE KNN maj-min
new_yeast None NB all
new_yeast None NB maj-min
new_yeast globalCS NB all
new_yeast globalCS NB maj-min
new_yeast SMOTE NB all
new_yeast SMOTE NB maj-min
thyroid-newthyroid None CART all
thyroid-newthyroid None CART maj-min
thyroid-newthyroid globalCS CART all
thyroid-newthyroid globalCS CART maj-min
thyroid-newthyroid SMOTE CART all
thyroid-newthyroid SMOTE CART maj-min
thyroid-newthyroid None KNN all
thyroid-newthyroid None KNN maj-min
thyroid-newthyroid globalCS KNN all
thyroid-newthyroid globalCS KNN maj-min
thyroid-newthyroid SMOTE KNN all
thyroid-newthyroid SMOTE KNN maj-min
thyroid-newthyroid None NB all
thyroid-newthyroid None NB maj-min
thyroid-newthyroid globalCS NB all
thyroid-newthyroid globalCS NB maj-min
thyroid-newthyroid SMOTE NB all
thyroid-newthyroid SMOTE NB maj-min


In [120]:
df_gmean = pd.DataFrame(results_g_mean)
df_acc = pd.DataFrame(results_acc)
df_avg_tpr = pd.DataFrame(results_avg_tpr)

for df in (df_gmean, df_acc, df_avg_tpr):
    df.set_index('dataset')
    df.columns = pd.MultiIndex.from_tuples(
        [('dataset', '')] + [(str(bc), str(bo), str(obs)) for bc in binary_classifiers for bo in binary_oversamplings for obs in oversample_between_strategies])
    
import os
import datetime
    
current_date = str(datetime.datetime.today())[:-7].replace(' ', '_').replace(':','_')

directory='OVO-'+current_date
os.makedirs(directory)

df_gmean.to_csv(f'./{directory}/ovo_kfold_gmean.csv', index=False)
df_acc.to_csv(f'./{directory}/ovo_kfold_acc.csv', index=False)
df_avg_tpr.to_csv(f'./{directory}/ovo_kfold_avg_tpr.csv', index=False)

In [121]:
df_gmean

dataset   CART                                          \
                             None         globalCS          SMOTE           
                       NaN    all maj-min      all maj-min    all maj-min   
0              1czysty-cut  0.940   0.940    0.945   0.947  0.954   0.953   
1           2delikatne-cut  0.663   0.674    0.690   0.678  0.704   0.694   
2             3mocniej-cut  0.430   0.428    0.378   0.407  0.435   0.467   
3   4delikatne-bezover-cut  0.729   0.739    0.750   0.749  0.779   0.782   
4            balance-scale  0.242   0.247    0.130   0.129  0.281   0.265   
5                cleveland  0.080   0.055    0.073   0.073  0.084   0.067   
6             cleveland_v2  0.050   0.047    0.043   0.066  0.084   0.050   
7                      cmc  0.431   0.429    0.419   0.425  0.442   0.422   
8              dermatology  0.937   0.913    0.919   0.921  0.913   0.908   
9                    glass  0.521   0.586    0.573   0.535  0.611   0.524   
10              hayes-roth  0.858   0.858    0.864   0.864  0.858   0.858   
11               new_ecoli  0.645   0.670    0.669   0.621  0.670   0.664   
12           new_led7digit  0.769   0.766    0.782   0.783  0.775   0.771   
13             new_vehicle  0.909   0.905    0.904   0.906  0.901   0.911   
14     new_winequality-red  0.232   0.242    0.219   0.241  0.315   0.337   
15               new_yeast  0.215   0.237    0.248   0.247  0.301   0.261   
16      thyroid-newthyroid  0.906   0.895    0.864   0.868  0.892   0.875   

      KNN                                             NB                   \
     None         globalCS          SMOTE           None         globalCS   
      all maj-min      all maj-min    all maj-min    all maj-min      all   
0   0.965   0.965    0.977   0.977  0.980   0.978  0.721   0.721    0.935   
1   0.666   0.666    0.726   0.705  0.699   0.702  0.430   0.430    0.803   
2   0.384   0.384    0.444   0.446  0.457   0.463  0.018   0.018    0.610   
3   0.757   0.757    0.803   0.803  0.804   0.817  0.653   0.653    0.877   
4   0.278   0.278    0.265   0.274  0.347   0.352  0.080   0.080    0.419   
5   0.040   0.040    0.068   0.078  0.139   0.087  0.096   0.096    0.145   
6   0.026   0.026    0.196   0.109  0.159   0.132  0.079   0.079    0.217   
7   0.460   0.460    0.459   0.460  0.469   0.468  0.484   0.484    0.467   
8   0.859   0.859    0.852   0.852  0.856   0.857  0.761   0.761    0.777   
9   0.490   0.490    0.525   0.504  0.518   0.498  0.207   0.207    0.199   
10  0.597   0.597    0.732   0.626  0.667   0.606  0.711   0.711    0.712   
11  0.741   0.741    0.757   0.752  0.758   0.745  0.287   0.287    0.315   
12  0.761   0.761    0.751   0.767  0.775   0.747  0.769   0.769    0.772   
13  0.863   0.863    0.883   0.881  0.878   0.879  0.453   0.453    0.501   
14  0.148   0.148    0.294   0.295  0.301   0.289  0.317   0.317    0.427   
15  0.248   0.248    0.403   0.362  0.388   0.360  0.066   0.066    0.057   
16  0.813   0.813    0.865   0.865  0.871   0.863  0.929   0.929    0.941   

                           
            SMOTE          
   maj-min    all maj-min  
0    0.935  0.938   0.934  
1    0.805  0.810   0.806  
2    0.583  0.616   0.583  
3    0.877  0.878   0.879  
4    0.419  0.436   0.442  
5    0.099  0.227   0.094  
6    0.084  0.283   0.083  
7    0.477  0.467   0.468  
8    0.761  0.779   0.761  
9    0.200  0.256   0.298  
10   0.719  0.702   0.720  
11   0.306  0.358   0.353  
12   0.770  0.757   0.758  
13   0.501  0.490   0.499  
14   0.428  0.412   0.413  
15   0.058  0.086   0.081  
16   0.941  0.938   0.943

In [122]:
df_acc

dataset   CART                                          \
                             None         globalCS          SMOTE           
                       NaN    all maj-min      all maj-min    all maj-min   
0              1czysty-cut  0.959   0.959    0.960   0.961  0.963   0.964   
1           2delikatne-cut  0.828   0.834    0.843   0.843  0.828   0.825   
2             3mocniej-cut  0.700   0.703    0.720   0.710  0.640   0.668   
3   4delikatne-bezover-cut  0.836   0.842    0.850   0.848  0.842   0.843   
4            balance-scale  0.540   0.543    0.526   0.519  0.497   0.529   
5                cleveland  0.568   0.545    0.561   0.567  0.525   0.551   
6             cleveland_v2  0.700   0.693    0.683   0.701  0.690   0.677   
7                      cmc  0.480   0.476    0.468   0.474  0.490   0.473   
8              dermatology  0.947   0.937    0.935   0.936  0.939   0.935   
9                    glass  0.716   0.740    0.733   0.701  0.697   0.716   
10              hayes-roth  0.838   0.838    0.844   0.844  0.838   0.838   
11               new_ecoli  0.770   0.788    0.784   0.769  0.770   0.773   
12           new_led7digit  0.790   0.786    0.792   0.793  0.792   0.789   
13             new_vehicle  0.915   0.910    0.906   0.910  0.898   0.911   
14     new_winequality-red  0.469   0.481    0.498   0.486  0.489   0.484   
15               new_yeast  0.509   0.511    0.508   0.517  0.515   0.509   
16      thyroid-newthyroid  0.947   0.942    0.928   0.935  0.942   0.930   

      KNN                                             NB                   \
     None         globalCS          SMOTE           None         globalCS   
      all maj-min      all maj-min    all maj-min    all maj-min      all   
0   0.973   0.973    0.963   0.963  0.970   0.967  0.903   0.903    0.878   
1   0.846   0.846    0.811   0.820  0.810   0.815  0.819   0.819    0.784   
2   0.731   0.731    0.636   0.659  0.624   0.635  0.705   0.705    0.572   
3   0.864   0.864    0.838   0.838  0.838   0.842  0.870   0.870    0.809   
4   0.728   0.728    0.629   0.641  0.636   0.637  0.732   0.732    0.613   
5   0.478   0.478    0.337   0.419  0.330   0.386  0.554   0.554    0.518   
6   0.683   0.683    0.535   0.538  0.476   0.497  0.673   0.673    0.624   
7   0.490   0.490    0.457   0.461  0.471   0.477  0.471   0.471    0.453   
8   0.880   0.880    0.866   0.866  0.872   0.876  0.872   0.872    0.874   
9   0.654   0.654    0.641   0.636  0.634   0.629  0.432   0.432    0.353   
10  0.643   0.643    0.734   0.650  0.665   0.634  0.700   0.700    0.694   
11  0.829   0.829    0.784   0.790  0.802   0.802  0.458   0.458    0.464   
12  0.786   0.786    0.765   0.783  0.798   0.784  0.790   0.790    0.790   
13  0.865   0.865    0.859   0.858  0.862   0.863  0.603   0.603    0.615   
14  0.445   0.445    0.408   0.427  0.399   0.418  0.533   0.533    0.468   
15  0.529   0.529    0.490   0.497  0.469   0.473  0.139   0.139    0.113   
16  0.921   0.921    0.921   0.921  0.919   0.919  0.967   0.967    0.963   

                           
            SMOTE          
   maj-min    all maj-min  
0    0.878  0.883   0.881  
1    0.791  0.800   0.800  
2    0.592  0.582   0.596  
3    0.809  0.821   0.820  
4    0.613  0.610   0.623  
5    0.544  0.539   0.549  
6    0.620  0.648   0.650  
7    0.464  0.453   0.457  
8    0.872  0.876   0.872  
9    0.358  0.371   0.393  
10   0.700  0.685   0.700  
11   0.458  0.502   0.494  
12   0.788  0.782   0.781  
13   0.615  0.611   0.611  
14   0.470  0.402   0.419  
15   0.114  0.155   0.156  
16   0.963  0.965   0.967

In [123]:
df_avg_tpr

dataset   CART                                          \
                             None         globalCS          SMOTE           
                       NaN    all maj-min      all maj-min    all maj-min   
0              1czysty-cut  0.941   0.941    0.946   0.948  0.955   0.954   
1           2delikatne-cut  0.708   0.718    0.730   0.727  0.736   0.728   
2             3mocniej-cut  0.510   0.511    0.509   0.507  0.493   0.525   
3   4delikatne-bezover-cut  0.769   0.775    0.785   0.784  0.801   0.804   
4            balance-scale  0.449   0.451    0.412   0.407  0.415   0.444   
5                cleveland  0.336   0.299    0.325   0.316  0.310   0.307   
6             cleveland_v2  0.347   0.351    0.298   0.333  0.373   0.337   
7                      cmc  0.451   0.448    0.440   0.446  0.461   0.443   
8              dermatology  0.940   0.921    0.925   0.925  0.924   0.921   
9                    glass  0.683   0.707    0.695   0.671  0.706   0.684   
10              hayes-roth  0.866   0.866    0.871   0.871  0.866   0.866   
11               new_ecoli  0.688   0.715    0.713   0.684  0.704   0.703   
12           new_led7digit  0.780   0.777    0.790   0.790  0.784   0.781   
13             new_vehicle  0.911   0.907    0.904   0.906  0.901   0.912   
14     new_winequality-red  0.346   0.357    0.366   0.366  0.383   0.391   
15               new_yeast  0.432   0.442    0.449   0.447  0.473   0.454   
16      thyroid-newthyroid  0.920   0.910    0.883   0.883  0.906   0.891   

      KNN                                             NB                   \
     None         globalCS          SMOTE           None         globalCS   
      all maj-min      all maj-min    all maj-min    all maj-min      all   
0   0.966   0.966    0.977   0.977  0.980   0.978  0.757   0.757    0.939   
1   0.715   0.715    0.747   0.738  0.727   0.732  0.567   0.567    0.811   
2   0.502   0.502    0.492   0.514  0.499   0.514  0.342   0.342    0.636   
3   0.797   0.797    0.825   0.825  0.825   0.832  0.735   0.735    0.885   
4   0.568   0.568    0.511   0.523  0.520   0.521  0.530   0.530    0.592   
5   0.227   0.227    0.233   0.244  0.243   0.245  0.329   0.329    0.332   
6   0.272   0.272    0.314   0.288  0.296   0.300  0.354   0.354    0.380   
7   0.470   0.470    0.463   0.464  0.470   0.473  0.495   0.495    0.491   
8   0.874   0.874    0.866   0.866  0.869   0.871  0.866   0.866    0.869   
9   0.638   0.638    0.663   0.649  0.659   0.644  0.501   0.501    0.489   
10  0.621   0.621    0.742   0.642  0.680   0.624  0.753   0.753    0.747   
11  0.764   0.764    0.773   0.767  0.772   0.761  0.513   0.513    0.534   
12  0.777   0.777    0.766   0.781  0.787   0.772  0.778   0.778    0.779   
13  0.865   0.865    0.886   0.884  0.880   0.881  0.582   0.582    0.601   
14  0.314   0.314    0.345   0.365  0.342   0.354  0.431   0.431    0.471   
15  0.478   0.478    0.487   0.480  0.486   0.475  0.324   0.324    0.310   
16  0.836   0.836    0.880   0.880  0.880   0.876  0.934   0.934    0.947   

                           
            SMOTE          
   maj-min    all maj-min  
0    0.939  0.941   0.937  
1    0.813  0.817   0.814  
2    0.616  0.640   0.617  
3    0.885  0.885   0.886  
4    0.592  0.585   0.603  
5    0.329  0.346   0.324  
6    0.352  0.396   0.357  
7    0.498  0.490   0.491  
8    0.866  0.870   0.866  
9    0.495  0.502   0.519  
10   0.753  0.740   0.753  
11   0.521  0.549   0.536  
12   0.778  0.766   0.766  
13   0.601  0.596   0.598  
14   0.474  0.462   0.458  
15   0.311  0.331   0.331  
16   0.947  0.944   0.949

In [124]:
ranks_gmean = dict()
ranks_acc = dict()
ranks_avg_tpr = dict()

for results, ranks in zip((results_g_mean, results_acc, results_avg_tpr), (ranks_gmean, ranks_acc, ranks_avg_tpr)):
    for ds_idx, ds in enumerate(datasets_names):
        ranks[ds] = dict()
        for bc in binary_classifiers:
            ranks[ds][bc] = rankdata(
                [-results[method][ds_idx] for method in methods if method[0]==bc])


In [145]:
avg_ranks_gmean=dict()
avg_ranks_acc=dict()
avg_ranks_avg_tpr=dict()

for avg_ranks, ranks in zip((avg_ranks_gmean, avg_ranks_acc, avg_ranks_avg_tpr), (ranks_gmean, ranks_acc, ranks_avg_tpr)):
    for bc in binary_classifiers:
        avg_ranks[bc]=dict()
        for method_idx, method in enumerate([m for m in methods if m[0]==bc]):
            avg_ranks[bc][method] = round(np.mean([ranks[ds][bc][method_idx] for ds in datasets_names]),3)
            
for avg_ranks in (avg_ranks_gmean, avg_ranks_acc, avg_ranks_avg_tpr):
    for bc in binary_classifiers:
        for method in methods:
            if method[0]==bc:
                avg_ranks[bc][str(method[1:])] = avg_ranks[bc][method]
                avg_ranks[bc].pop(method)

df1 = pd.DataFrame(avg_ranks_gmean).T
df2 = pd.DataFrame(avg_ranks_acc).T
df3 = pd.DataFrame(avg_ranks_avg_tpr).T


In [146]:
avg_ranks_gmean

{'CART': {"(None, 'all')": 3.971,
  "(None, 'maj-min')": 4.059,
  "('globalCS', 'all')": 4.059,
  "('globalCS', 'maj-min')": 3.647,
  "('SMOTE', 'all')": 2.147,
  "('SMOTE', 'maj-min')": 3.118},
 'KNN': {"(None, 'all')": 4.941,
  "(None, 'maj-min')": 4.941,
  "('globalCS', 'all')": 3.0,
  "('globalCS', 'maj-min')": 3.176,
  "('SMOTE', 'all')": 1.941,
  "('SMOTE', 'maj-min')": 3.0},
 'NB': {"(None, 'all')": 4.735,
  "(None, 'maj-min')": 4.735,
  "('globalCS', 'all')": 3.059,
  "('globalCS', 'maj-min')": 3.088,
  "('SMOTE', 'all')": 2.559,
  "('SMOTE', 'maj-min')": 2.824}}

In [147]:
avg_ranks_avg_tpr

{'CART': {"(None, 'all')": 3.765,
  "(None, 'maj-min')": 3.676,
  "('globalCS', 'all')": 3.5,
  "('globalCS', 'maj-min')": 4.029,
  "('SMOTE', 'all')": 2.853,
  "('SMOTE', 'maj-min')": 3.176},
 'KNN': {"(None, 'all')": 4.529,
  "(None, 'maj-min')": 4.529,
  "('globalCS', 'all')": 3.059,
  "('globalCS', 'maj-min')": 2.853,
  "('SMOTE', 'all')": 2.941,
  "('SMOTE', 'maj-min')": 3.088},
 'NB': {"(None, 'all')": 4.559,
  "(None, 'maj-min')": 4.559,
  "('globalCS', 'all')": 3.029,
  "('globalCS', 'maj-min')": 3.235,
  "('SMOTE', 'all')": 2.706,
  "('SMOTE', 'maj-min')": 2.912}}

## Mean ranks in g-mean:

In [149]:
df1

,"(None, 'all')","(None, 'maj-min')","('globalCS', 'all')","('globalCS', 'maj-min')","('SMOTE', 'all')","('SMOTE', 'maj-min')"
CART,3.971,4.059,4.059,3.647,2.147,3.118
KNN,4.941,4.941,3.000,3.176,1.941,3.000
NB,4.735,4.735,3.059,3.088,2.559,2.824


## Mean ranks in accuracy:

In [150]:
df2

,"(None, 'all')","(None, 'maj-min')","('globalCS', 'all')","('globalCS', 'maj-min')","('SMOTE', 'all')","('SMOTE', 'maj-min')"
CART,3.353,3.382,3.353,2.912,3.882,4.118
KNN,1.794,1.794,4.618,3.971,4.588,4.235
NB,2.353,2.353,4.647,4.324,3.971,3.353


## Mean ranks in average accuracy (average recall):

In [151]:
df3

,"(None, 'all')","(None, 'maj-min')","('globalCS', 'all')","('globalCS', 'maj-min')","('SMOTE', 'all')","('SMOTE', 'maj-min')"
CART,3.765,3.676,3.500,4.029,2.853,3.176
KNN,4.529,4.529,3.059,2.853,2.941,3.088
NB,4.559,4.559,3.029,3.235,2.706,2.912
